## **Given Task 1**


1. Multi-Agent System Architecture
    - Design a scalable architecture for a multi-agent system where agents can be dynamically created, assigned tasks, and collaborate.
    - Implement a prototype of this system using Python, incorporating at least one LLM (e.g., GPT-3.5 or GPT-4) for agent decision-making.
    -  Develop a mechanism for agents to communicate and share information.
    -  Build to support at least one use case (more if possible). You can choose a use case of your choice or select from any of the following domains; marketing agent, sales agent, accounting agent, HR agent, Software Engineering agent.

#### **Solution** - Software Engineering Spark agent

In [1]:
%%capture

!pip install -r requirements.txt

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
api_base_secret = os.getenv('api_base_secret')
azure_api_key_secret = os.getenv('azure_api_key_secret')

In [4]:
config_list = [{
    "engine": "gpt-35-turbo-16k", 
    "model": "gpt-35-turbo-16k",
    "api_base": api_base_secret, # Your Secret Base URL Endpoint :)
    "api_type": "azure", 
    "api_version": "2023-07-01-preview",
    "api_key": azure_api_key_secret # Your Secret Azure API key :)
}]

In [5]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
import chromadb
import autogen
# from tqdm.autonotebook import tqdm, trange


llm_config = {
    "request_timeout": 120,
    "seed": 21,
    "config_list": config_list,
    "temperature": 0.1,
}

autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The boss who ask questions and give tasks.",
    code_execution_config=False,  # we don't want to execute code in this case.
)

boss_aid = RetrieveUserProxyAgent(
    name="Boss_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "chunk_token_size": 1500,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="owais_experiments/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

owais_coder = AssistantAgent(
    name="Senior_Data_Scientist_Owais",
    is_termination_msg=termination_msg,
    system_message="You are a senior Data Scientist and a Python Engineer. Your Name is Owais. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

reviewer = autogen.AssistantAgent(
    name="Code_Reviewer",
    is_termination_msg=termination_msg,
    system_message="You are a code reviewer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

PROBLEM = "How to use spark for parallel training in FLAML? Give me sample code."

def _reset_agents():
    boss.reset()
    boss_aid.reset()
    owais_coder.reset()
    pm.reset()
    reviewer.reset()

In [6]:
def rag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss_aid, owais_coder, pm, reviewer], messages=[], max_round=30
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    boss_aid.initiate_chat(
        manager,
        problem=PROBLEM,
        n_results=5,)
    
    
rag_chat()

Trying to create collection.


/Users/owaisahmad/Desktop/Agentic Worker/env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/owaisahmad/Desktop/Agentic Worker/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


doc_ids:  [['doc_0', 'doc_1']]
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
Boss_Assistant (to chat_manager):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How to use spark for parallel training in FLAML? Give me sample code.

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:

- Use Spark ML estimators for AutoML.
- Use Spark to run training in parallel spark jobs.

## Spark ML Estimators

FLAML integrates estimator

In [7]:
def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss, owais_coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,)
norag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
Senior_Data_Scientist_Owais (to chat_manager):

To use Spark for parallel training in FLAML, you need to follow these steps:

Step 1: Install FLAML and PySpark
First, make sure you have FLAML and PySpark installed. You can install them using pip:

```
pip install flaml pyspark
```

Step 2: Import the necessary libraries
Next, import the required libraries in your Python script:

```python
from flaml import AutoML
from pyspark.sql import SparkSession
```

Step 3: Create a SparkSession
Create a SparkSession object to connect to your Spark cluster:

```python
spark = SparkSession.builder \
    .appName("FLAML Example") \
    .getOrCreate()
```

Step 4: Load your data into a Spark DataFrame
Load your data into a Spark DataFrame. You can use the `spark.read` method to read data from various sources such as CSV, Parqu